<a href="https://colab.research.google.com/github/aaron123-hash/aaronez/blob/master/New_Cases_and_Deaths_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install pyPDF package
%%capture
!pip install PyPDF2

# Import libraries
import re
import PyPDF2
import pandas as pd

In [2]:
def extract_county_cases(text):
  
  #For counties with 2 names, change it to one name.
  text = re.sub(r'\band\b','',text) 
  text = re.sub(r'[()]?[,]?[.]?\.?','',text) 
  text = text.replace("Homa Bay", "HomaBay")
  text = text.replace("West Pokot","WestPokot")
  text = text.replace("Tharaka Nithi ","TharakaNithi ")
  text = text.replace("Murang’a","Muranga")
  #Tana 
  text = text.replace("Tana River","TanaRiver")
  #Taita/Taveta
  text = text.replace("Taita Taveta","TaitaTaveta")
  #Trans Nzoia
  text = text.replace("Trans Nzoia","TransNzoia")
  #Uasin Gishu
  text = text.replace("Uasin Gishu","UasinGishu")
  #Elgeyo/Marakwet
  text = text.replace("Elgeyo Marakwet","ElgeyoMarakwet")
  per_county21= text.split()
  #print(per_county21)
  
  county_data = pd.DataFrame()
  county_data['Names'] = per_county21
  county_data['count'] = county_data['Names']

  #county_data.loc[county_data.Names == county_data.count, "count"] = ''
  # df.iloc[::2]
  just_county = county_data.iloc[::2].reset_index()
  just_county.drop('index',axis = 1, inplace= True)
  just_county
  vals = county_data.iloc[1::2].reset_index()
  just_county['count'] = vals['count']
  return just_county

In [3]:
# remove parenthesis
def parenthesis(text):
  import re
  text=text.strip()
  #text = re.sub(r'Distribution of positive cases by age is as follows; ', '', text)
  text = re.sub(r'[a-zA-Z\s;]', '',text)
  text=re.sub(r'[\n]+', ' ',text)
  text=re.sub(r'[\s]+', '',text)
  text = re.sub(r'[()\s.]',' ',text)
  text = re.sub(r'[,]','',text)
  text = re.sub(r'\\n\s+',' ',text)
  text = re.sub(r'years',' ',text)
  text = re.sub(r'and',' ',text)
  text = re.sub(r'above',' ',text)
  return text

In [4]:
# importing required modules 
%%capture
import PyPDF2
import re

def clean(text):
  # text = re.sub("Murang’a",'Muranga',text)
  # text = re.sub('^\n\d[\n\s?\n?]*','',text)
  
  text = re.sub('\n*','',text)
  return text    
# creating a pdf file object 
pdfFileObj = open('/content/drive/MyDrive/Data Science/data_pdfs/Press-Statement-on-Covid-19-10th-August-2021.pdf', 'rb') 
    
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
pdf = []   
# printing number of pages in pdf file 
print(pdfReader.numPages) 
i = 0
for i in range(0,pdfReader.numPages):    
  # creating a page object 
  pageObj = pdfReader.getPage(i) 
  i +=1   
  # extracting text from page 
  # import re
  extract = clean(pageObj.extractText())
  pdf.append(extract)
  i +=1
  # extract = extract.strip()
  # extract=re.sub(r'[\s]+', '',extract)
  print(pageObj.extractText())
  # print(extract)
  full_text = ' '.join(pdf)  
# closing the pdf file object 
pdfFileObj.close()


FileNotFoundError: ignored

In [5]:
# County distribution
def county_distribution(extracted_text):
  start = extracted_text.find('In terms of County distribution; ') + 33
  end = extracted_text.find('.', start)
  new_cases_percounty = extracted_text[start:end]
  # new_cases_percounty = parenthesis(new_cases_percounty)
  new_cases_percounty = extract_county_cases(new_cases_percounty)
  new_cases_percounty.rename(columns={'Names': 'Age_group', 'count': 'New_cases_percounty'}, inplace = True)
  return new_cases_percounty
  # return extracted_text[start:end]

county_distribution(full_text)

NameError: ignored

In [ ]:
# positive cases by age
def age_positive(text):
  start = text.find('Distribution of positive cases by age is as follows; ') + 53
  end = text.find('.', start)
  new_cases = text[start:end]
  new_cases = parenthesis(new_cases)
  new_cases = extract_county_cases(new_cases)
  new_cases.rename(columns={'Names': 'Age_group', 'count': 'New_cases'}, inplace = True)
  return new_cases
  # return text[start:end]

age_positive(full_text)

In [ ]:
# Deaths by age
def deaths_age(text):
  start = text.find('New deaths by age; ') + 19
  end = text.find('.', start)
  total_deaths = text[start:end]
  total_deaths = parenthesis(total_deaths)
  total_deaths = extract_county_cases(total_deaths)
  total_deaths.rename(columns={'Names': 'Age_group', 'count': 'Fatalities'}, inplace = True)
  return total_deaths

deaths_age(full_text)